## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/docs/guides/evaluation/examples/comparisons).

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***

**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).


In [ ]:
#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain Specs: "Hello World" -> retrieval_chain
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
# context_getter = RunnableLambda(lambda x: x)  ## TODO
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain Specs: retrieval_chain -> generator_chain
##   -> {"output" : <str>, ...} -> output_puller
# generator_chain = RunnableLambda(lambda x: x)  ## TODO
generator_chain = chat_prompt | llm  ## TODO
generator_chain = {"output" : generator_chain } | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

<br>

### **Step 4: [Exercise]** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [ ]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    rag_answer = ""
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')
